# week 2

## 데이터 전처리

### 데이터 전처리 정의

- 데이터의 품질을 올리는 과정
- 데이터의 품질은 데이터 분석의 90%를 좌우한다.

### 데이터 전처리 과정

- 데이터 실수화: 컴퓨터가 이해할 수 있는 값으로의 변환
- 불완전한 데이터 제거: NULL, NA, NAN 값의 제거
- 잡음 섞인 데이터 제거
    - 가격 데이터에 있는 (-) 값 제거
    - 연령 데이터중 과도하게 큰 값 제거 or 소수점 제거
        - ex) 나이 값으로 200, 300 등의 값이 존재하는 경우
- 모순된 데이터 제거: 남성 데이터 중 주민번호가 '2'로 시작하는 경우
- 불균형 데이터 해결
    - 과소표집(undersampling), 과대표집(oversampling)

### 데이터 전처리의 주요 기법

1. 데이터 실수화 (Data Vectorization)
    - 범주형 자료, 텍스트 자료, 이미지 자료 등을 실수로 구성된 형태로 전환하는 것
    - 2차원 자료의 예시
        - [n_sample, n_features]
        - 2차원 자료는 행렬 혹은 2차원 텐서라 불림
    - 자료의 유형
        - 연속형 자료
        - 텍스트 자료
        - 범주형 자료
            - One-hot encoding을 이용한 데이터 실수화
            

In [ ]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False) # sparse=False 로 하면 데이터를 눈으로 볼 수 있음
x=[{'city':'seoul','temp':10.0}, {'city':'Dubai', 'temp':33.5}, {'city':'LA','temp':20.0}]
x

[{'city': 'seoul', 'temp': 10.0},
 {'city': 'Dubai', 'temp': 33.5},
 {'city': 'LA', 'temp': 20.0}]

* 희소행렬(Sparse Matrix)
    * 행렬의 값이 대부분 0인 경우를 가리키는 표현
    * 희소 행렬은 프로그램 시 불필요한 0 값으로 인해 메모리 낭비가 심함
    * 행렬의 크기가 커서 연산시 시간도 많이 소모됨
    * COO 표현식과 CSR 표현식을 통해 문제 해결 가능

In [ ]:
vec.fit_transform(x) # x를 범주형 수량화 자료로 변환

array([[ 0. ,  0. ,  1. , 10. ],
       [ 1. ,  0. ,  0. , 33.5],
       [ 0. ,  1. ,  0. , 20. ]])

In [ ]:
text=['떴다 떴다 비행기 날아라 날아라','높이 높이 날아라 우리 비행기','내가 만든 비행기 날아라 날아라','멀리 멀리 날아라 우리 비행기']
text

['떴다 떴다 비행기 날아라 날아라',
 '높이 높이 날아라 우리 비행기',
 '내가 만든 비행기 날아라 날아라',
 '멀리 멀리 날아라 우리 비행기']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec2 = CountVectorizer() # default는 sparse=True
t=vec2.fit_transform(text).toarray() # toarray() 한 이유는 sparse=True를 풀고 text를 수량화 배열 자료로 변환
import pandas as pd
t1=pd.DataFrame(t, columns=vec2.get_feature_names())
t1

,날아라,내가,높이,떴다,만든,멀리,비행기,우리
0,2,0,0,2,0,0,1,0
1,1,0,2,0,0,0,1,1
2,2,1,0,0,1,0,1,0
3,1,0,0,0,0,2,1,1


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid=TfidfVectorizer()
x2=tfid.fit_transform(text).toarray() # 높은 빈도는 낮은 가중치, 낮은 빈도는 높은 가중치
x3=pd.DataFrame(x2,columns=tfid.get_feature_names())
x3

,날아라,내가,높이,떴다,만든,멀리,비행기,우리
0,0.450735,0.000000,0.00000,0.86374,0.000000,0.00000,0.225368,0.000000
1,0.229589,0.000000,0.87992,0.00000,0.000000,0.00000,0.229589,0.346869
2,0.569241,0.545415,0.00000,0.00000,0.545415,0.00000,0.284620,0.000000
3,0.229589,0.000000,0.00000,0.00000,0.000000,0.87992,0.229589,0.346869



2. 데이터 정제 (Data Cleaning)
    - 없는 데이터는 채우고, 잡음 데이터는 제거하고, 모순 데이터를 올바른 데이터로 교정하는 것
    - 결측 데이터 채우기 (Empty Values)
        - 결측 데이터: np.nan, npNaN, None
        - 평균(mean), 중위수(median), 최빈수(most frequent value)로 대처하는 기법 사용
        - 사용가능함수
            - sklearn의 Imputer(): 입력인자로 평균, 중위수, 최빈수 선택


In [ ]:
import numpy as np
x_miss = np.array([[1,2,3,None],[5,np.NAN,7,8],[None,10,11,12],[13,np.nan,15,16]])
x_miss

array([[1, 2, 3, None],
       [5, nan, 7, 8],
       [None, 10, 11, 12],
       [13, nan, 15, 16]], dtype=object)

In [ ]:
from sklearn.impute import SimpleImputer
im = SimpleImputer(strategy='mean') # 평균 값으로 대처
im.fit(x_miss)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)


3. 데이터 통합 (Data Integration)
    - 여러 개의 데이터 파일을 하나로 합치는 과정
    - pandas의 merge() 함수 사용   

In [ ]:
#www.kaggle.com 으로 들어간 후 새로운 account를 생성하고
#www.kaggle.com/c/rossmann-store-sales/data 에서 자료를 다운받는다.
#zip을 풀어서 적절한 폴더(이경우 머신러닝교재)
import pandas as pd
df1=pd.read_csv(\"train.csv\",engine='python')
print(df1.shape)
type(df1)

In [ ]:
df1.head()

In [ ]:
df2=pd.read_csv(\"store.csv\",engine='python')
df2.shape

In [ ]:
df=pd.merge(df1,df2,on='Store')
df.shape


4. 데이터 축소 (Data Reduction)
    - 데이터가 과도하게 큰 경우, 분석 및 학습에 시간이 오래 걸리고 비효율적이기 떄문에 데이터의 수를 줄이거나 (sampling), 데이터 차원을 축소하는 작업
5. 데이터 변환 (Data Transformation)
    - 데이터를 정규화 하거나, 로그를 씌우거나, 평균값을 계산하여 사용하거나, 사람 나이 등을 10대, 20대, 30대 등으로 구간 화 하는 작업
    - 머신러닝 알고리즘은 데이터가 가진 특성(Feature)들을 비교하여 데이터 패턴을 찾음
    - 데이터가 가진 특성 간 스케일 차이가 심하면 패턴을 찾는데 문제가 발생함
    ![image](https://user-images.githubusercontent.com/41141851/111131702-dabb0400-85bb-11eb-9e58-491fb4fe87b1.png)
    - 정규화가 표준화보다 유용함. 단, 데이터 특성이 bell-shape 이거나 이상치가 있을 경우에는 표준화가 유용함
6. 데이터 균형 & 불균형 (Data Balancing)
    - 특정 클래스의 관측치가 다른 클래스에 비해 매우 낮을 경우 샘플링을 통해 비율을 맞추는 작업
    - 머신러닝의 목적이 분류 일 떄, 특정 클래스의 관측치가 다른 클래스에 비해 매우 낮게 나타나면 이러한 자료를 불균형자료라고 함
    ![image](https://user-images.githubusercontent.com/41141851/111131548-b3643700-85bb-11eb-9ff7-89b08ec87b61.png)
    - 데이터 불균형 해소 기법
        - 과소표집(undersampling), 과대표집(oversampling)
        - 일반적으로 과소표집보다 과대표집이 통계적으로 유용함
        - 의사결정나무(decision tree)와 앙상블(ensemble)은 상대적으로 불균형자료에 강인한 특성을 보임